Paramètrage notebook

In [10]:
#importation package
import pandas as pd
import duckdb

#config affichage
pd.set_option("display.max_columns", None)  # Affiche toutes les colonnes sans limitation
pd.set_option("display.max_colwidth", None)  # Affiche la largeur complète des colonnes sans couper le texte
pd.set_option("display.expand_frame_repr", False)  # Empêche le repliement des colonnes lors de l'affichage du DataFrame

# Connexion
from pipelines.tasks.config.common import DUCKDB_FILE
con = duckdb.connect(database=DUCKDB_FILE, read_only=True)

In [11]:
#affichage tables & vues
con.sql('show tables')

┌──────────────────────────────────┐
│               name               │
│             varchar              │
├──────────────────────────────────┤
│ ana__resultats_communes          │
│ cog_communes                     │
│ edc_communes                     │
│ edc_prelevements                 │
│ edc_resultats                    │
│ int__lien_cdreseau_refreneceprel │
│ int__lien_commune_cdreseau       │
│ int__mapping_category_simple     │
│ int__prelevements_uniques        │
│ int__resultats_udi_communes      │
│ laposte_communes                 │
│ mapping_categories               │
│ stg_communes__cog                │
│ stg_communes__laposte            │
│ stg_edc__communes                │
│ stg_edc__prevelevements          │
│ stg_edc__resultats               │
├──────────────────────────────────┤
│             17 rows              │
└──────────────────────────────────┘

Objectif : Création d'un modèle dbt pour le résultat des nitrites  
Tâches : création d'un fichier qui remprend le resultat du dernier prélèvement contenant les colonnes 
- cdreseau
- période
- catégorie
- résultat
- date

Catégorie Nitrates
3 paramètres à regarder:  
nitrates (en no3)  
nitrites (en no2)  
nitrates/50 + nitrites/3  

2 situations:  
Nitrates < 50 mg/L et nitrites < 0,5 mg/L et nitrate/50 + nitrites/3 < 1 mg/L (eau conforme)  
Nitrates > 50 mg/L et/ou nitrites > 0,5 mg/L et/ou nitrate/50 + nitrites/3 > 1 mg/L (eau non conforme) (fait passer l’affichage total polluant en rouge)


Filtre de la table sur les catégories nitrites

In [12]:
#paramètres à regarder NO2, NO3
query_nitrates = """ 
SELECT *, ROW_NUMBER() OVER(PARTITION BY cdreseau, cdparametresiseeaux  ORDER BY datetimeprel DESC, valtraduite DESC) AS row_number
FROM int__resultats_udi_communes
WHERE cdparametresiseeaux IN ('NO2','NO3')
AND CURRENT_DATE - datetimeprel < INTERVAL 1 YEAR
ORDER BY cdreseau, cdparametresiseeaux
  """

nitrates = con.sql(query_nitrates).df()
nitrates

,referenceprel,cdparametresiseeaux,valtraduite,limitequal,de_partition,limitequal_float,unite,categorie,cdreseau,inseecommune,datetimeprel,row_number
0,00100143925,NO2,0.00,"<=0,5 mg/L",2025,0.5,mg/L,nitrite,001000003,01007,2025-01-21 12:35:00,1
1,00100143925,NO3,14.00,<=50 mg/L,2025,50.0,mg/L,nitrite,001000003,01007,2025-01-21 12:35:00,1
2,00100143918,NO2,0.00,"<=0,5 mg/L",2025,0.5,mg/L,nitrite,001000241,01439,2025-01-17 11:10:00,1
3,00100143918,NO2,0.00,"<=0,5 mg/L",2025,0.5,mg/L,nitrite,001000241,01139,2025-01-17 11:10:00,2
4,00100143918,NO2,0.00,"<=0,5 mg/L",2025,0.5,mg/L,nitrite,001000241,01050,2025-01-17 11:10:00,3
...,...,...,...,...,...,...,...,...,...,...,...,...
73513,97400140947,NO3,0.71,<=50 mg/L,2025,50.0,mg/L,nitrite,974004294,97411,2025-01-09 08:35:00,1
73514,97400140966,NO2,0.00,"<=0,1 mg/L",2025,0.1,mg/L,nitrite,974004295,97418,2025-01-20 07:47:00,1
73515,97400140966,NO3,3.40,<=50 mg/L,2025,50.0,mg/L,nitrite,974004295,97418,2025-01-20 07:47:00,1
73516,97400140948,NO2,0.00,"<=0,1 mg/L",2025,0.1,mg/L,nitrite,974004298,97420,2025-01-13 09:00:00,1


Récupération des derniers résultats

In [13]:
#dernier prélèvement
query_dernier_prel = """ 
SELECT *
FROM nitrates
WHERE row_number = 1
 """
dernier_prel = con.sql(query_dernier_prel).df()
dernier_prel


,referenceprel,cdparametresiseeaux,valtraduite,limitequal,de_partition,limitequal_float,unite,categorie,cdreseau,inseecommune,datetimeprel,row_number
0,00100143925,NO2,0.00,"<=0,5 mg/L",2025,0.5,mg/L,nitrite,001000003,01007,2025-01-21 12:35:00,1
1,00100143925,NO3,14.00,<=50 mg/L,2025,50.0,mg/L,nitrite,001000003,01007,2025-01-21 12:35:00,1
2,00100143918,NO2,0.00,"<=0,5 mg/L",2025,0.5,mg/L,nitrite,001000241,01439,2025-01-17 11:10:00,1
3,00100143918,NO3,3.30,<=50 mg/L,2025,50.0,mg/L,nitrite,001000241,01402,2025-01-17 11:10:00,1
4,00100143923,NO2,0.00,"<=0,5 mg/L",2025,0.5,mg/L,nitrite,001000248,01004,2025-01-21 12:10:00,1
...,...,...,...,...,...,...,...,...,...,...,...,...
11823,97400140947,NO3,0.71,<=50 mg/L,2025,50.0,mg/L,nitrite,974004294,97411,2025-01-09 08:35:00,1
11824,97400140966,NO2,0.00,"<=0,1 mg/L",2025,0.1,mg/L,nitrite,974004295,97418,2025-01-20 07:47:00,1
11825,97400140966,NO3,3.40,<=50 mg/L,2025,50.0,mg/L,nitrite,974004295,97418,2025-01-20 07:47:00,1
11826,97400140948,NO2,0.00,"<=0,1 mg/L",2025,0.1,mg/L,nitrite,974004298,97420,2025-01-13 09:00:00,1


Aggrégation des résultats  : Table intermédiaire  
2 situations:  
Nitrates < 50 mg/L et nitrites < 0,5 mg/L et nitrate/50 + nitrites/3 < 1 mg/L (eau conforme)  
Nitrates > 50 mg/L et/ou nitrites > 0,5 mg/L et/ou nitrate/50 + nitrites/3 > 1 mg/L (eau non conforme) (fait passer l’affichage total polluant en rouge)

In [14]:
#définition des résultats
query_resultats_nitrites = """ 

SELECT referenceprel, cdreseau, MAX(datetimeprel) AS datetimeprel,
SUM(CASE
    WHEN cdparametresiseeaux = 'NO2' THEN valtraduite
    ELSE 0
END) AS valtraduite_2,

SUM(CASE
    WHEN cdparametresiseeaux = 'NO3' THEN valtraduite
    ELSE 0
END) AS valtraduite_3,

ROUND((valtraduite_2/50 + valtraduite_3/3),2) AS valtraduite_NO2_NO3

FROM dernier_prel
GROUP BY referenceprel, cdreseau
 """
resultats_nitrites = con.sql(query_resultats_nitrites).df()
resultats_nitrites


,referenceprel,cdreseau,datetimeprel,valtraduite_2,valtraduite_3,valtraduite_NO2_NO3
0,00100143877,001000258,2025-01-21 09:45:00,0.0,0.00,0.00
1,00100143931,001000268,2025-01-13 09:57:00,0.0,6.00,2.00
2,00100143998,001000278,2025-01-22 11:24:00,0.0,13.00,4.33
3,00100144051,001000289,2025-01-21 11:14:00,0.0,0.00,0.00
4,00100143912,001000293,2025-01-28 11:29:00,0.0,3.70,1.23
...,...,...,...,...,...,...
7584,97400141070,974000781,2025-01-06 09:45:00,0.0,0.86,0.29
7585,97400141255,974001245,2025-01-13 10:35:00,0.0,0.00,0.00
7586,97400141301,974003623,2025-01-30 08:10:00,0.0,4.10,1.37
7587,97400141307,974004288,2025-01-30 08:45:00,0.0,0.35,0.12


Création de la table avec le résultat final 

2 situations:  
Nitrates < 50 mg/L et nitrites < 0,5 mg/L et nitrate/50 + nitrites/3 < 1 mg/L (eau conforme)  
Nitrates > 50 mg/L et/ou nitrites > 0,5 mg/L et/ou nitrate/50 + nitrites/3 > 1 mg/L (eau non conforme) (fait passer l’affichage total polluant en rouge)  

La table d'origine présente des 'valtraduite' Null => traduit par 'aucun résultat' dans la table finale : ==> est-ce que c'est OK ou faut t'il enlever les prélèvements à valtraduite NULL?

In [15]:

query_resultat_nitrite_dernier = """

SELECT cdreseau, referenceprel, 'dernier relevé' AS periode, 'nitrites' AS categorie, 
CASE 
    WHEN valtraduite_2 < 50 AND valtraduite_3 < 0.5 AND valtraduite_NO2_NO3 < 1
    THEN 'eau conforme'
    WHEN valtraduite_2 >= 50 OR valtraduite_3 >= 0.5 OR valtraduite_NO2_NO3 >= 1
    THEN 'eau non conforme'
    ELSE 'aucun résultat'
END AS resultat ,
datetimeprel
FROM resultats_nitrites
ORDER BY datetimeprel
 """
resultat_nitrite_dernier = con.sql(query_resultat_nitrite_dernier).df()
resultat_nitrite_dernier


,cdreseau,referenceprel,periode,categorie,resultat,datetimeprel
0,086000622,08600133560,dernier relevé,nitrites,eau non conforme,2025-01-02 08:47:00
1,026000675,02600171794,dernier relevé,nitrites,eau non conforme,2025-01-02 08:51:00
2,028001276,02800125276,dernier relevé,nitrites,eau non conforme,2025-01-02 09:00:00
3,086000333,08600133561,dernier relevé,nitrites,eau non conforme,2025-01-02 09:03:00
4,083001260,08300290363,dernier relevé,nitrites,eau non conforme,2025-01-02 09:04:00
...,...,...,...,...,...,...
7584,034001311,03400327052,dernier relevé,nitrites,aucun résultat,2025-01-31 14:36:00
7585,062000735,06200288259,dernier relevé,nitrites,eau non conforme,2025-01-31 15:34:00
7586,062004264,06200288259,dernier relevé,nitrites,eau non conforme,2025-01-31 15:34:00
7587,072000572,07200139826,dernier relevé,nitrites,eau non conforme,2025-01-31 15:37:00


FIN CREATION MODELE

---------------------------------------------

EXPLO VERIF RESULTAT

In [16]:
#vérification des valtraduite NULL dans la table d'origine
con.sql('SELECT * FROM dernier_prel WHERE valtraduite IS NULL' ).df()

,referenceprel,cdparametresiseeaux,valtraduite,limitequal,de_partition,limitequal_float,unite,categorie,cdreseau,inseecommune,datetimeprel,row_number
0,03400327115,NO3,NaN,<=50 mg/L,2025,50.0,mg/L,nitrite,034000005,34041,2025-01-10 11:32:00,1
1,03400327103,NO3,NaN,<=50 mg/L,2025,50.0,mg/L,nitrite,034000006,34056,2025-01-07 10:44:00,1
2,03400327108,NO3,NaN,<=50 mg/L,2025,50.0,mg/L,nitrite,034000009,34182,2025-01-10 09:58:00,1
3,03400327090,NO3,NaN,<=50 mg/L,2025,50.0,mg/L,nitrite,034000012,34042,2025-01-27 11:19:00,1
4,03400327116,NO3,NaN,<=50 mg/L,2025,50.0,mg/L,nitrite,034000021,34051,2025-01-28 10:01:00,1
...,...,...,...,...,...,...,...,...,...,...,...,...
173,03400327135,NO3,NaN,<=50 mg/L,2025,50.0,mg/L,nitrite,034007938,34148,2025-01-07 10:55:00,1
174,03400327259,NO3,NaN,<=50 mg/L,2025,50.0,mg/L,nitrite,034008030,34256,2025-01-28 12:36:00,1
175,03400327145,NO3,NaN,<=50 mg/L,2025,50.0,mg/L,nitrite,034008215,34191,2025-01-07 13:24:00,1
176,03400327269,NO3,NaN,<=50 mg/L,2025,50.0,mg/L,nitrite,034008225,34179,2025-01-28 13:50:00,1


In [27]:
#vérification des nobmre de cdreseau >1 dans le résultat final
verif_double_cdreseau = con.sql('select  cdreseau, COUNT(*) AS nbcdreseau FROM resultat_nitrite_dernier GROUP BY cdreseau HAVING nbcdreseau>1 ORDER BY nbcdreseau DESC')
verif_double_cdreseau

┌───────────┬────────────┐
│ cdreseau  │ nbcdreseau │
│  varchar  │   int64    │
├───────────┼────────────┤
│ 051000483 │          2 │
│ 017000223 │          2 │
│ 017000286 │          2 │
│ 037004035 │          2 │
│ 006000892 │          2 │
│ 034000520 │          2 │
│ 036000333 │          2 │
│ 061000369 │          2 │
│ 089000570 │          2 │
│ 028000977 │          2 │
│     ·     │          · │
│     ·     │          · │
│     ·     │          · │
│ 017000095 │          2 │
│ 028000792 │          2 │
│ 086000325 │          2 │
│ 037000152 │          2 │
│ 085000108 │          2 │
│ 007000694 │          2 │
│ 062004262 │          2 │
│ 051000802 │          2 │
│ 089000542 │          2 │
│ 032003919 │          2 │
├───────────┴────────────┤
│  469 rows (20 shown)   │
└────────────────────────┘

In [41]:
#vérif pourquoi il y a des UDI en double
con.sql('SELECT * FROM dernier_prel WHERE cdreseau = \'051000802\'')

#==> parce que les paramètres analysés sont issues de deux prélèvements différents

┌───────────────┬─────────────────────┬─────────────┬────────────┬──────────────┬──────────────────┬─────────┬───────────┬───────────┬──────────────┬─────────────────────┬────────────┐
│ referenceprel │ cdparametresiseeaux │ valtraduite │ limitequal │ de_partition │ limitequal_float │  unite  │ categorie │ cdreseau  │ inseecommune │    datetimeprel     │ row_number │
│    varchar    │       varchar       │   double    │  varchar   │    int16     │      float       │ varchar │  varchar  │  varchar  │   varchar    │      timestamp      │   int64    │
├───────────────┼─────────────────────┼─────────────┼────────────┼──────────────┼──────────────────┼─────────┼───────────┼───────────┼──────────────┼─────────────────────┼────────────┤
│ 05100139910   │ NO2                 │         0.0 │ <=0,5 mg/L │         2025 │              0.5 │ mg/L    │ nitrite   │ 051000802 │ 51279        │ 2025-01-08 10:45:00 │          1 │
│ 05100139886   │ NO3                 │         0.0 │ <=50 mg/L  │         

In [61]:
#VERIFICATION écart de date entre les prélèvements
con.sql(' SELECT cdreseau, MAX(datetimeprel) - MIN(datetimeprel)  AS ecartprel  FROM dernier_prel GROUP BY cdreseau HAVING ecartprel <> INTERVAL \'0 days\' ORDER BY ecartprel DESC')

#==> est-ce que ce n'est pas un problème pour le calcul de la conformité s'il y a 27 jours d'écart entre le prélèvement de NO2 et NO3


┌───────────┬──────────────────┐
│ cdreseau  │    ecartprel     │
│  varchar  │     interval     │
├───────────┼──────────────────┤
│ 076000358 │ 27 days 23:25:00 │
│ 076001796 │ 27 days 22:40:00 │
│ 076000359 │ 25 days 22:05:00 │
│ 044000170 │ 24 days 01:00:00 │
│ 017000286 │ 24 days 00:27:00 │
│ 062003743 │ 24 days 00:20:00 │
│ 062000528 │ 24 days 00:20:00 │
│ 062004262 │ 23 days 23:39:00 │
│ 017000489 │ 23 days 23:34:00 │
│ 085000186 │ 23 days 22:30:00 │
│     ·     │    ·             │
│     ·     │    ·             │
│     ·     │    ·             │
│ 086000516 │ 00:12:00         │
│ 089000517 │ 00:12:00         │
│ 058000330 │ 00:11:00         │
│ 042000021 │ 00:09:00         │
│ 086000683 │ 00:06:00         │
│ 086000367 │ 00:05:00         │
│ 086000335 │ 00:04:00         │
│ 086000325 │ 00:04:00         │
│ 086000467 │ 00:04:00         │
│ 086000359 │ 00:04:00         │
├───────────┴──────────────────┤
│     466 rows (20 shown)      │
└──────────────────────────────┘

In [62]:
#vérif des referenceprel > 1 ==> un prélèvements pour plusieurs UDI
con.sql('SELECT referenceprel, COUNT(*) AS countref FROM dernier_prel WHERE cdreseau IN (SELECT DISTINCT cdreseau FROM verif_double_cdreseau) GROUP BY referenceprel HAVING countref>1')


┌───────────────┬──────────┐
│ referenceprel │ countref │
│    varchar    │  int64   │
├───────────────┼──────────┤
│ 00700205382   │        2 │
│ 00700205330   │        3 │
│ 00700205338   │        2 │
│ 01600123424   │        2 │
│ 01600123452   │        4 │
│ 01700149554   │        2 │
│ 01700149384   │        6 │
│ 02800125370   │        2 │
│ 02800125453   │        2 │
│ 02800125334   │        2 │
│      ·        │        · │
│      ·        │        · │
│      ·        │        · │
│ 05700174664   │        6 │
│ 06400216576   │        2 │
│ 07600297777   │        4 │
│ 07600298416   │        2 │
│ 08500472676   │        2 │
│ 08600133651   │        2 │
│ 08600133650   │        2 │
│ 08900133526   │        4 │
│ 08900133502   │        3 │
│ 09400169298   │        3 │
├───────────────┴──────────┤
│    79 rows (20 shown)    │
└──────────────────────────┘

In [40]:
con.sql('SELECT * FROM dernier_prel WHERE referenceprel = \'01600123465\'')

┌───────────────┬─────────────────────┬─────────────┬────────────┬──────────────┬──────────────────┬─────────┬───────────┬───────────┬──────────────┬─────────────────────┬────────────┐
│ referenceprel │ cdparametresiseeaux │ valtraduite │ limitequal │ de_partition │ limitequal_float │  unite  │ categorie │ cdreseau  │ inseecommune │    datetimeprel     │ row_number │
│    varchar    │       varchar       │   double    │  varchar   │    int16     │      float       │ varchar │  varchar  │  varchar  │   varchar    │      timestamp      │   int64    │
├───────────────┼─────────────────────┼─────────────┼────────────┼──────────────┼──────────────────┼─────────┼───────────┼───────────┼──────────────┼─────────────────────┼────────────┤
│ 01600123465   │ NO3                 │         8.7 │ <=50 mg/L  │         2025 │             50.0 │ mg/L    │ nitrite   │ 016000331 │ 16134        │ 2025-01-29 10:53:00 │          1 │
│ 01600123465   │ NO3                 │         8.7 │ <=50 mg/L  │         